In [13]:
from tensorflow.keras import backend as K
K.clear_session()

In [14]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model

vgg16 = load_model('vgg_model.keras')

feature_extractor = tf.keras.Model(
    inputs=vgg16.input,
    outputs=vgg16.get_layer("global_average_pooling2d").output
)

In [15]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import cv2
import tensorflow as tf

# Klinik veri
clinical_df = pd.read_csv('processed_clinical_with_diagnosis.csv')

folder_path = 'Readformlp'
clinical_ids = clinical_df['cases.submitter_id'].tolist()

features = []
file_ids = []

for fname in os.listdir(folder_path):
    if fname.endswith('.npy'):
        case_id = fname[:12]
        if case_id in clinical_ids:
            arr = np.load(os.path.join(folder_path, fname))  # (N, 240, 240)
            
            # Eğer slice sayısı 2500'den küçükse geç
            if arr.shape[0] < 2500:
                continue
            # Eğer büyükse ilk 2500'ü al
            elif arr.shape[0] > 2500:
                arr = arr[:2500]

            # 1. Resize ve 3 kanal ekle
            imgs_resized = np.zeros((2500, 224, 224, 3), dtype=np.float32)
            for i in range(2500):
                resized = cv2.resize(arr[i], (224, 224))
                imgs_resized[i] = np.stack([resized]*3, axis=-1)
            
            # 2. Toplu predict
            feature_vectors = feature_extractor.predict(imgs_resized, batch_size=64, verbose=1)  # (2500, 512) (2500,3)
            
            features.append(feature_vectors)
            file_ids.append(case_id)




2025-05-17 03:40:03.787753: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.40GiB (rounded to 1505280000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-05-17 03:40:03.787770: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-05-17 03:40:03.787775: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 48, Chunks in use: 48. 12.0KiB allocated for chunks. 12.0KiB in use in bin. 1.2KiB client-requested in use in bin.
2025-05-17 03:40:03.787776: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 16, Chunks in use: 16. 8.5KiB allocated for chunks. 8.5KiB in use in bin. 8.0KiB client-requested in use in bin.
2025-

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

/framework/bfc_allocator.cc:1114] InUse at 78f820000800 of size 256 next 5
2025-05-17 03:40:03.787818: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 78f820000900 of size 256 next 7
2025-05-17 03:40:03.787818: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 78f820000a00 of size 256 next 8
2025-05-17 03:40:03.787819: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 78f820000b00 of size 256 next 6
2025-05-17 03:40:03.787820: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 78f820000c00 of size 256 next 9
2025-05-17 03:40:03.787821: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 78f820000d00 of size 256 next 14
2025-05-17 03:40:03.787821: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 78f820000e00 of size 256 next 12
2025-05-17 03:40:03.787822: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 78f820000f00 of size 512 next 13
2025-